# Import modules and load data

In [3]:
# import really important modules
import numpy as np
import pandas as pd

In [4]:
# get a sample of our harvest data so that we can play around
harvest = r"C:\Users\steph\Documents\Python\Projects\Silas\Scripts\RandomForest_training.csv"

# Data cleanup
The harvest data needs to be reorganized and cleaned up. The harvest data is stored in a csv file. Instead of manipulating the data with excel, I'm going to test my hand at Pandas and Numpy. Apparently they're all the rage these days. 

In [5]:
# set the harvest data to the object fish
fish = pd.read_csv(harvest)
# create dataframe with weather variables
df = pd.DataFrame(fish)

# view top 5 rows
df.head()

,binomial_nomenclature,common_name,UTC,date,year,month,day,dayofyear,latitude,longitude,...,wind_speed_kmph,pressure_mb,dewpoint_f,head_index_f,percipitation_mm,visualbility_km,moon_phase,angler,angler_id,harvest_id
0,Acantharchus pomotis,Mud Sunfish,2016-10-30 18:13:00 UTC,20161030,2016,10,30,304,38.113966,-77.261811,...,11,1015,56,70,0.0,10,0.001377,pmk00001,204678,5540659
1,Acantharchus pomotis,Mud Sunfish,2015-09-27 20:25:00 UTC,20150927,2015,9,27,270,38.059625,-77.203420,...,16,1024,65,74,0.2,10,0.998815,pmk00001,204678,6098682
2,Acantharchus pomotis,Mud Sunfish,2017-05-21 15:22:00 UTC,20170521,2017,5,21,141,38.072827,-77.371777,...,9,1024,56,63,0.2,10,0.234520,pmk00001,204678,6520177
3,Acantharchus pomotis,Mud Sunfish,2017-07-22 17:19:00 UTC,20170722,2017,7,22,203,40.193045,-74.273730,...,10,1011,72,92,25.6,9,0.007514,pmk00001,204678,7517746
4,Acantharchus pomotis,Mud Sunfish,2016-04-22 22:05:14 UTC,20160422,2016,4,22,113,30.723598,-82.488785,...,12,1017,62,73,5.1,9,0.994536,levfrid,187197,8751263


In [6]:
# drop fields from dataframe that we do not need
df = df.drop(['common_name', 'UTC', 'date', 'year', 'latitude',
              'longitude', 'angler', 'angler_id', 'harvest_id'], axis=1)

df.head()

,binomial_nomenclature,month,day,dayofyear,cloud_cover,temp_f,humidity,wind_direction_degrees,wind_speed_kmph,pressure_mb,dewpoint_f,head_index_f,percipitation_mm,visualbility_km,moon_phase
0,Acantharchus pomotis,10,30,304,40,81,64,219,11,1015,56,70,0.0,10,0.001377
1,Acantharchus pomotis,9,27,270,90,80,80,80,16,1024,65,74,0.2,10,0.998815
2,Acantharchus pomotis,5,21,141,86,68,77,109,9,1024,56,63,0.2,10,0.234520
3,Acantharchus pomotis,7,22,203,35,94,70,122,10,1011,72,92,25.6,9,0.007514
4,Acantharchus pomotis,4,22,113,52,85,75,217,12,1017,62,73,5.1,9,0.994536


# Prepare and create test and trainig datasets 
This is where we create some test and training datasets for the 
random forest regression model. Test data is the actual harvest
data, while train data is a mix of test data plus the random data
we created in the previous step

# More practice with Pandas
In order to become more familar with Pandas and dataframes, I'm going to try to figure out the range of values for each weather variable.

I might be able to use these ranges later to generate a realistic subsample of the harvest data with using randomly generated values for each weather variable. The random values need to be within the normal range of the real data. We may use this subsample as the training dataset for the random forest regression module.... I don't really know at this point.

In [7]:
# create variables to hold the min and max values for specific columns
# use iloc to select by label, aka header or column
var_max = df.iloc[:, 4:15].max(axis = 0)
var_min = df.iloc[:, 4:15].min(axis = 0)

In [8]:
# view min values
var_min

cloud_cover                  7.000000
temp_f                      55.000000
humidity                    56.000000
wind_direction_degrees      26.000000
wind_speed_kmph              4.000000
pressure_mb               1006.000000
dewpoint_f                  31.000000
head_index_f                51.000000
percipitation_mm             0.000000
visualbility_km              7.000000
moon_phase                   0.001377
dtype: float64

In [9]:
# view min values
var_max

cloud_cover                100.000000
temp_f                      96.000000
humidity                    88.000000
wind_direction_degrees     326.000000
wind_speed_kmph             22.000000
pressure_mb               1029.000000
dewpoint_f                  74.000000
head_index_f                94.000000
percipitation_mm            25.600000
visualbility_km             10.000000
moon_phase                   0.998993
dtype: float64

## Okay! 
So first I isolated the labels for which I wanted range values for by using `df.iloc[:, 4:15]`. Then I attached `.max(axis = 0)` at the end, which returns the max value for each column (axis 1 is columns, axis 0 is rows). 

Now let's create new dataframes to hold our min and max values.

In [10]:
# create dataframe from min values
dfmin = pd.DataFrame(var_min)

# view the new dataframe
dfmin

,0
cloud_cover,7.000000
temp_f,55.000000
humidity,56.000000
wind_direction_degrees,26.000000
wind_speed_kmph,4.000000
pressure_mb,1006.000000
dewpoint_f,31.000000
head_index_f,51.000000
percipitation_mm,0.000000
visualbility_km,7.000000


In [11]:
# create dataframe from max values
dfmax = pd.DataFrame(var_max)

# view the new dataframe
dfmax

,0
cloud_cover,100.000000
temp_f,96.000000
humidity,88.000000
wind_direction_degrees,326.000000
wind_speed_kmph,22.000000
pressure_mb,1029.000000
dewpoint_f,74.000000
head_index_f,94.000000
percipitation_mm,25.600000
visualbility_km,10.000000


## Hmmmm. 
So this is nice and all, I have two dataframes with minimum and maximum values. 

But I would like to have the max and min values in one dataframe, maybe side by side so it's easy to identify the range. Maybe I'll make a list using the **var_min** and **var_max** variables.

In [12]:
# create list of min and max values
d = ([var_min, var_max])

# view the list of values
d

[cloud_cover                  7.000000
 temp_f                      55.000000
 humidity                    56.000000
 wind_direction_degrees      26.000000
 wind_speed_kmph              4.000000
 pressure_mb               1006.000000
 dewpoint_f                  31.000000
 head_index_f                51.000000
 percipitation_mm             0.000000
 visualbility_km              7.000000
 moon_phase                   0.001377
 dtype: float64, cloud_cover                100.000000
 temp_f                      96.000000
 humidity                    88.000000
 wind_direction_degrees     326.000000
 wind_speed_kmph             22.000000
 pressure_mb               1029.000000
 dewpoint_f                  74.000000
 head_index_f                94.000000
 percipitation_mm            25.600000
 visualbility_km             10.000000
 moon_phase                   0.998993
 dtype: float64]

In [14]:
# create dataframe with the max and min list
df2 = pd.DataFrame(d, index = ['Minimum', 'Maximum'])

# view the dataframe of min and max values
df2.head()

,cloud_cover,temp_f,humidity,wind_direction_degrees,wind_speed_kmph,pressure_mb,dewpoint_f,head_index_f,percipitation_mm,visualbility_km,moon_phase
Minimum,7.0,55.0,56.0,26.0,4.0,1006.0,31.0,51.0,0.0,7.0,0.001377
Maximum,100.0,96.0,88.0,326.0,22.0,1029.0,74.0,94.0,25.6,10.0,0.998993


## Well then. That's not exactly what I wanted. But close!
I want the max and min values as the headers (columns, labels, whatever). Let's try to figure this out.

I know the **dfmin** and **dfmax** dataframes look the way I want (max and min in columns, not rows), but I need to combime them into one dataframe.

In [15]:
# create dataframe using dfmin
#nindex = dfmin.columns[:11]
df3 = pd.DataFrame(dfmin)

# view the new dataframe
df3

,0
cloud_cover,7.000000
temp_f,55.000000
humidity,56.000000
wind_direction_degrees,26.000000
wind_speed_kmph,4.000000
pressure_mb,1006.000000
dewpoint_f,31.000000
head_index_f,51.000000
percipitation_mm,0.000000
visualbility_km,7.000000


### This looks good.
Now I need to add the maximum values from **dfmax** and also add some labels

In [16]:
# add Min & Max labels to dataframe, populate with values from dfmin/dfmax 1st column
df3['Minimum'] = dfmin[0]
df3['Maximum'] = dfmax[0]

In [17]:
# view the new dataframe
df3

,0,Minimum,Maximum
cloud_cover,7.000000,7.000000,100.000000
temp_f,55.000000,55.000000,96.000000
humidity,56.000000,56.000000,88.000000
wind_direction_degrees,26.000000,26.000000,326.000000
wind_speed_kmph,4.000000,4.000000,22.000000
pressure_mb,1006.000000,1006.000000,1029.000000
dewpoint_f,31.000000,31.000000,74.000000
head_index_f,51.000000,51.000000,94.000000
percipitation_mm,0.000000,0.000000,25.600000
visualbility_km,7.000000,7.000000,10.000000


In [18]:
# drop "0" label from database
df3 = df3.drop(0, 1)
# view the dataframe of min and max values
df3

,Minimum,Maximum
cloud_cover,7.000000,100.000000
temp_f,55.000000,96.000000
humidity,56.000000,88.000000
wind_direction_degrees,26.000000,326.000000
wind_speed_kmph,4.000000,22.000000
pressure_mb,1006.000000,1029.000000
dewpoint_f,31.000000,74.000000
head_index_f,51.000000,94.000000
percipitation_mm,0.000000,25.600000
visualbility_km,7.000000,10.000000
